In [14]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

In [27]:
# Carrega arquivo parquet
df = pd.read_parquet("DB/List-dependencies.parquet")

In [30]:
df

,Project Name,Dependency Name
0,a4g,a4g:client
1,a4g,a4g:server
2,anchovy,derelict-ft
3,anchovy,dlib
4,anchovy,derelict-fi
...,...,...
18967354,jh-lib,lodash-es
18967355,jh-lib,jh-utils
18967356,jh-lib,classnames
18967357,jh-lib,antd


In [28]:
graph = nx.from_pandas_edgelist(
    df,
    source="Project Name",
    target="Dependency Name",
    create_using=nx.DiGraph(),
)

In [35]:
import numpy as np
import networkx as nx
from collections import Counter
import time

def analyze_graph_complete_large_scale(graph):
    """
    Análise completa e detalhada de um grafo de dependências de grande escala,
    otimizada para performance e relevância das métricas.
    """
    
    print("="*60)
    print("          ANÁLISE COMPLETA DO GRAFO DE DEPENDÊNCIAS")
    print("="*60)
    
    start_time = time.time()
    
    # ============= ESTATÍSTICAS BÁSICAS =============
    print("\n🔍 ESTATÍSTICAS BÁSICAS")
    print("-" * 40)
    num_nodes = graph.number_of_nodes()
    num_edges = graph.number_of_edges()
    
    print(f"Nós: {num_nodes}")
    print(f"Arestas: {num_edges}")
    if num_nodes > 0:
        density = nx.density(graph)
        print(f"Densidade: {f'{density:.8f}'.replace('.', ',')}")
    print(f"Tipo: {'Direcionado' if graph.is_directed() else 'Não-direcionado'}")
    
    # ============= ANÁLISE DE CONECTIVIDADE (Métricas Globais) =============
    print("\n🔗 ANÁLISE DE CONECTIVIDADE")
    print("-" * 40)
    
    # Estas operações são eficientes e podem ser feitas no grafo completo
    degrees = dict(graph.degree())
    in_degrees = dict(graph.in_degree())
    out_degrees = dict(graph.out_degree())
    
    degree_values = list(degrees.values())
    in_degree_values = list(in_degrees.values())
    out_degree_values = list(out_degrees.values())
    
    print(f"Grau médio: {f'{np.mean(degree_values):.2f}'.replace('.', ',')}")
    print(f"Grau máximo: {max(degree_values)}")
    print(f"Mediana do grau: {f'{np.median(degree_values):.2f}'.replace('.', ',')}")
    
    print(f"\nIn-degree médio (popularidade): {f'{np.mean(in_degree_values):.2f}'.replace('.', ',')}")
    print(f"In-degree máximo: {max(in_degree_values)}")
    print(f"Out-degree médio (complexidade): {f'{np.mean(out_degree_values):.2f}'.replace('.', ',')}")
    print(f"Out-degree máximo: {max(out_degree_values)}")
    
    # ============= COMPONENTES CONECTADOS (Estrutura Macro) =============
    print("\n🌐 COMPONENTES CONECTADOS")
    print("-" * 40)
    
    # Focar no maior componente conectado é a melhor estratégia para grafos grandes
    print("Identificando o maior ecossistema (componente fracamente conectado)...")
    weakly_connected = list(nx.weakly_connected_components(graph))
    largest_wcc = max(weakly_connected, key=len)
    main_graph = graph.subgraph(largest_wcc)
    
    print(f"Número de ecossistemas (componentes fracamente conectados): {len(weakly_connected)}")
    print(f"Tamanho do maior ecossistema: {len(largest_wcc)} nós ({f'{100 * len(largest_wcc) / num_nodes:.2f}'.replace('.', ',')}% do total)")
    
    # ============= TOP NÓS (Hubs e Autoridades) =============
    print("\n⭐ TOP 15 DEPENDÊNCIAS MAIS POPULARES (MAIOR IN-DEGREE)")
    print("-" * 40)
    top_dependencies = sorted(in_degrees.items(), key=lambda x: x[1], reverse=True)[:15]
    for i, (dep, count) in enumerate(top_dependencies, 1):
        print(f"{i:2d}. {dep}: {count} projetos dependem")
    
    print("\n📦 TOP 15 PROJETOS COM MAIS DEPENDÊNCIAS (MAIOR OUT-DEGREE)")
    print("-" * 40)
    top_projects = sorted(out_degrees.items(), key=lambda x: x[1], reverse=True)[:15]
    for i, (proj, count) in enumerate(top_projects, 1):
        print(f"{i:2d}. {proj}: {count} dependências")
        
    # ============= ANÁLISE DE NÓS ESTRUTURAIS =============
    print("\n📊 ANÁLISE DE TIPOS DE NÓS")
    print("-" * 40)
    
    isolated_nodes = [n for n, d in degrees.items() if d == 0]
    print(f"Nós isolados (sem conexões): {len(isolated_nodes)}")
    
    source_nodes = [n for n, d in in_degrees.items() if d == 0 and out_degrees.get(n, 0) > 0]
    print(f"Nós fonte (projetos que não são dependência de ninguém): {len(source_nodes)}")
    
    sink_nodes = [n for n, d in out_degrees.items() if d == 0 and in_degrees.get(n, 0) > 0]
    print(f"Nós sumidouro (dependências que não possuem outras dependências): {len(sink_nodes)}")

    # ============= MÉTRICAS DE CENTRALIDADE (Amostra Inteligente) =============
    print("\n🎯 ANÁLISE DE CENTRALIDADE (amostra de 5000 nós do maior ecossistema)")
    print("-" * 40)
    
    sample_size = min(5000, len(main_graph.nodes()))
    sample_nodes = np.random.choice(list(main_graph.nodes()), sample_size, replace=False)
    sample_graph = main_graph.subgraph(sample_nodes)
    
    try:
        # PageRank: Mede a influência de um nó na rede. Rápido e eficaz.
        print("Calculando PageRank (influência)...")
        pagerank = nx.pagerank(sample_graph, alpha=0.85)
        top_pagerank = sorted(pagerank.items(), key=lambda x: x[1], reverse=True)[:5]
        print("Top 5 por PageRank:")
        for node, score in top_pagerank:
            print(f"  {node}: {f'{score:.6f}'.replace('.', ',')}")

        # Centralidade de Intermediação: Mede a importância de um nó como "ponte".
        # Usa-se uma aproximação (k) para ser viável em grafos grandes.
        print("\nCalculando Centralidade de Intermediação (importância como ponte)...")
        k_betweenness = min(200, len(sample_graph.nodes()))
        betweenness = nx.betweenness_centrality(sample_graph, k=k_betweenness, normalized=True)
        top_betweenness = sorted(betweenness.items(), key=lambda x: x[1], reverse=True)[:5]
        print(f"Top 5 por Intermediação (amostra k={k_betweenness}):")
        for node, score in top_betweenness:
            print(f"  {node}: {f'{score:.6f}'.replace('.', ',')}")
            
    except Exception as e:
        print(f"Erro ao calcular centralidade: {e}")
    
    # ============= ANÁLISE DE CICLOS (Amostra) =============
    print("\n🔄 ANÁLISE DE CICLOS (amostra de 20.000 nós)")
    print("-" * 40)
    
    try:
        # A detecção de ciclos no grafo inteiro é computacionalmente inviável.
        # Uma amostra grande nos dá uma boa ideia da presença e tamanho dos ciclos.
        cycle_sample_size = min(20000, len(main_graph.nodes()))
        cycle_sample_nodes = np.random.choice(list(main_graph.nodes()), cycle_sample_size, replace=False)
        cycle_subgraph = main_graph.subgraph(cycle_sample_nodes)
        
        print(f"Detectando ciclos em amostra de {cycle_sample_size} nós...")
        # Limita a busca para não travar em grafos muito densos
        cycles = list(islice(nx.simple_cycles(cycle_subgraph), 1000))
        
        if cycles:
            print(f"Ciclos simples encontrados na amostra: {len(cycles)} (limitado a 1000)")
            cycle_lengths = [len(c) for c in cycles]
            print(f"Tamanho médio dos ciclos: {f'{np.mean(cycle_lengths):.2f}'.replace('.', ',')}")
            print(f"Maior ciclo encontrado na amostra: {max(cycle_lengths)} nós")
        else:
            print("Nenhum ciclo encontrado na amostra. O grafo é majoritariamente acíclico.")

    except Exception as e:
        print(f"Erro na análise de ciclos: {e}")
    
    # ============= RESUMO FINAL =============
    elapsed_time = time.time() - start_time
    print("\n" + "="*60)
    print(f"✅ ANÁLISE COMPLETADA EM {f'{elapsed_time:.2f}'.replace('.', ',')} SEGUNDOS")
    print("="*60)

# --- Para executar a análise ---
# Certifique-se de que a variável 'graph' está definida na sua sessão do notebook
from itertools import islice

if 'graph' in locals():
    analyze_graph_complete_large_scale(graph)
else:
    print("Erro: A variável 'graph' não foi encontrada. Por favor, crie o grafo primeiro.")


          ANÁLISE COMPLETA DO GRAFO DE DEPENDÊNCIAS

🔍 ESTATÍSTICAS BÁSICAS
----------------------------------------
Nós: 1758429
Arestas: 14078099
Densidade: 0,00000455
Tipo: Direcionado

🔗 ANÁLISE DE CONECTIVIDADE
----------------------------------------
Grau médio: 16,01
Grau máximo: 193045
Mediana do grau: 5,00

In-degree médio (popularidade): 8,01
In-degree máximo: 192961
Out-degree médio (complexidade): 8,01
Out-degree máximo: 1000

🌐 COMPONENTES CONECTADOS
----------------------------------------
Identificando o maior ecossistema (componente fracamente conectado)...
Número de ecossistemas (componentes fracamente conectados): 11301
Tamanho do maior ecossistema: 1604174 nós (91,23% do total)

⭐ TOP 15 DEPENDÊNCIAS MAIS POPULARES (MAIOR IN-DEGREE)
----------------------------------------
 1. mocha: 192961 projetos dependem
 2. eslint: 177661 projetos dependem
 3. typescript: 128561 projetos dependem
 4. chai: 117227 projetos dependem
 5. webpack: 116054 projetos dependem
 6. babel-